# Chapter 3 Exercises

## Set Up

In [ ]:
# file paths
data_root = "https://raw.githubusercontent.com/kmerkl22/k-s-ISL/main/Data/"
image_root = "https://github.com/kmerkl22/k-s-ISL/raw/main/Images/"

## Problem 8

This question involves the use of simple linear regression on the Auto
data set.

### Part a
Use the sm.OLS() function to perform a simple linear regression
with mpg as the response and horsepower as the predictor. Use
the summarize() function to print the results. Comment on the
output. For example:
1.   Is there a relationship between the predictor and the response?
2.   How strong is the relationship between the predictor and
the response?
3.   Is the relationship between the predictor and the response
positive or negative?
4.   What is the predicted mpg associated with a horsepower of
98? What are the associated 95 % confdence and prediction
intervals?